In [8]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from Preprocessing import load_images,resize_images
from keras.utils import np_utils
%matplotlib inline

In [9]:
tk.Tk().withdraw()
path_Train=tkFileDialog.askdirectory()
print path_Train

/home/edward/project /Untitled Folder


In [4]:
X_Train=load_images(path_Train)
X_Train=resize_images(X_Train)

In [5]:
X_Train=np.array(X_Train)
X_Train.shape

(1000, 64, 64, 3)

In [6]:
tk.Tk().withdraw()
path_Test=tkFileDialog.askdirectory()
print path_Test

/home/edward/project /Untitled Folder 2


In [10]:
X_Test=load_images(path_Test)
X_Test=resize_images(X_Test)

In [11]:
X_Test=np.array(X_Test)
X_Test.shape

(300, 64, 64, 3)

In [13]:
data_Train=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Test_v2_Part3_GroundTruth.csv")

In [14]:
data_Train=data_Train.iloc[0:1000,1]
data_Test=data_Test.iloc[0:300,1]
print data_Train.shape
print data_Test.shape

(1000,)
(300,)


In [15]:
y_Train=data_Train
y_test=data_Test

In [ ]:
y_Train=np.array(y_Train)
y_Test=np.array(y_test)

In [16]:
learning_rate=0.001
training_epoch=15
batch_size=100
display_step=1

n_hidden_1=256
n_hidden_2=256
#n_hidden_3=256
#n_hidden_4=256
n_input=64*64*3
n_classes=2

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])

In [ ]:
def NeuralNet(x,weights,biases):
  
    

In [17]:
def multiplayer_perceptron(x,weights,biases):
    # first layer
    #with tf.device('/gpu:0'):
    #x=tf.reshape(x,shape=[-1,64,64,1])
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    # second layer
    #with tf.device('/gpu:1'):
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    # third layer
    """
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.relu(layer_3)
    # forth layer
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.relu(layer_4)
    # output layer

    """
    out_layer=tf.matmul(layer_2,weights['out'])+biases['out']
    return out_layer

In [18]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    #'h3':tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3])),
    #'h4':tf.Variable(tf.random_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_hidden_1,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    #'b3':tf.Variable(tf.random_normal([n_hidden_3])),
    #'b4':tf.Variable(tf.random_normal([n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred=multiplayer_perceptron(x,weights,biases)

# define loss and optimizer
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initilizing the variables
init=tf.global_variables_initializer()

In [ ]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training 
    
    for epoch in range(training_epoch):
        avg_cost=0
        for i in range(batch_size):
            train_idx=np.random.randint(X_Train.shape[0],size=100)
            batch_x_Train=X_Train[train_idx,:]
            batch_y_Train=y_Train[train_idx]
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x_Train,y:batch_y_Train})
            avg_cost+=c/batch_size
            
        if epoch%display_step==0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished"
    
    Predictions=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(Predictions,"float"))
    test_idx=np.random.randint(X_Test.shape[0],size=100)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    print "Training Accuracy:" ,accuracy.eval({x:batch_x_Train,y:batch_y_Train})*100
    print "Testing Accuracy:" ,accuracy.eval({x:batch_x_Test,y:batch_y_Test})*100